In [18]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [19]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.externals.six import StringIO  
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics import mean_squared_error,confusion_matrix, classification_report

In [27]:
# Import logistic regression function
from sklearn.linear_model import LogisticRegression
# calculate accuracy measures and confusion matrix
from sklearn import metrics

In [28]:
test = pd.read_csv('test.csv')

In [29]:
citi = pd.read_csv('train.csv')
citi.columns

Index(['Ref.No', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'Y'], dtype='object')

In [30]:
test=test.loc[:,['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7']]
test.head()

,X1,X2,X3,X4,X5,X6,X7
0,50000,2,1,2,25,0,0
1,120000,2,2,2,24,0,0
2,120000,2,2,2,27,1,-2
3,60000,1,1,2,28,0,-1
4,100000,2,1,2,28,3,2


In [31]:
citi.columns=['Ref_No','credit_amount','gender','education','marital_status','age','sep_hist','aug_hist','Y']
test.columns=['credit_amount','gender','education','marital_status','age','sep_hist','aug_hist']
citi.columns

Index(['Ref_No', 'credit_amount', 'gender', 'education', 'marital_status',
       'age', 'sep_hist', 'aug_hist', 'Y'],
      dtype='object')

In [32]:
citi.info()
# No null values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21600 entries, 0 to 21599
Data columns (total 9 columns):
Ref_No            21600 non-null int64
credit_amount     21600 non-null int64
gender            21600 non-null int64
education         21600 non-null int64
marital_status    21600 non-null int64
age               21600 non-null int64
sep_hist          21600 non-null int64
aug_hist          21600 non-null int64
Y                 21600 non-null int64
dtypes: int64(9)
memory usage: 1.5 MB


In [33]:
df=citi.copy()

In [34]:
y = df[['Y']]
X = df.drop(['Y','Ref_No'], axis = 1)

In [35]:
X.columns

Index(['credit_amount', 'gender', 'education', 'marital_status', 'age',
       'sep_hist', 'aug_hist'],
      dtype='object')

In [36]:
#dummy_df=X[['gender', 'education', 'marital_status', 'sep_hist', 'aug_hist']]
#X = X.drop(['gender', 'education', 'marital_status', 'sep_hist', 'aug_hist'], axis = 1)

In [37]:
#dummy_df=pd.get_dummies(dummy_df, prefix=['gender', 'edu', 'marital_status', 'sep', 'aug'], columns=['gender', 'education', 'marital_status', 'sep_hist', 'aug_hist'], drop_first=True)
#dummy_df.head()

In [38]:
X.shape

(21600, 7)

In [39]:
#X = pd.concat([X, dummy_df], axis=1, join='inner')

In [40]:
X.shape

(21600, 7)

In [41]:
def classficationModelAccuracies(x, y, model):
    model1 = model
    ## split data into train and test
    seed = 10
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = seed)
    ## Train the model with train data
    model1.fit(x_train, y_train)
    
    ## Train accuracies
    predictedClass = model1.predict(x_train)
    cMatrix = metrics.confusion_matrix(predictedClass, y_train)
    cMatrix
    # Classification accuracy
    trainClassificationAccuracy = (cMatrix[0][0] + cMatrix[1][1])/sum(sum(cMatrix))
    ## TP percentage
    trainTpPct = (cMatrix[1][1])/(cMatrix[1][0] + cMatrix[1][1])
    ## FP = Flase postive -> does not have disease but predicted as having disease
    # FP percentage
    trainFpPct = (cMatrix[0][1])/(cMatrix[0][0] + cMatrix[0][1])
    trainAccuracies = [len(y_train), trainClassificationAccuracy, trainTpPct, trainFpPct]
    
    ## Test accuracies
    predictedClass = model1.predict(x_test)
    cMatrix = metrics.confusion_matrix(predictedClass, y_test)
    cMatrix
    # Classification accuracy
    testClassificationAccuracy = (cMatrix[0][0] + cMatrix[1][1])/sum(sum(cMatrix))
    ## TP percentage
    testTpPct = (cMatrix[1][1])/(cMatrix[1][0] + cMatrix[1][1])
    ## FP = Flase postive -> does not have disease but predicted as having disease
    # FP percentage
    testFpPct = (cMatrix[0][1])/(cMatrix[0][0] + cMatrix[0][1])
    testAccuracies = [len(y_test), testClassificationAccuracy, testTpPct, testFpPct]
    
    ## Results
    resultDf = pd.DataFrame(index = ["dataSize", "accuracy", "tpPct", "fpPct"])
    resultDf['train'] = trainAccuracies
    resultDf['test'] = testAccuracies
    
    return(round(resultDf, 4))

In [42]:
model=LogisticRegression()
classficationModelAccuracies(X, y, model)

C:\Users\hp\Anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in longlong_scalars
  app.launch_new_instance()
C:\Users\hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in longlong_scalars


,train,test
dataSize,15120.0000,6480.0000
accuracy,0.7771,0.7741
tpPct,NaN,NaN
fpPct,0.2229,0.2259


In [43]:
predited_values=model.predict(test)

In [44]:
solution=pd.DataFrame()
solution["Ref.No"]=list(range(1,2400))
solution["Y"]=list(predited_values)

In [45]:
solution.to_csv("solution_basic_logistic.csv", index=False)

### Decission Tree

In [46]:
from sklearn.tree import DecisionTreeClassifier
#DecisionTreeClassifier is the corresponding Classifier
DTClassifier = DecisionTreeClassifier (max_depth=5, random_state = 0,max_features=6 , criterion='entropy')

In [47]:
model=DTClassifier
classficationModelAccuracies(X, y, model)

,train,test
dataSize,15120.0000,6480.0000
accuracy,0.8220,0.8114
tpPct,0.6971,0.6671
fpPct,0.1620,0.1704


In [48]:
predited_values=model.predict(test)
solution=pd.DataFrame()
solution["Ref.No"]=list(range(1,2400))
solution["Y"]=list(predited_values)

In [49]:
solution.head()

,Ref.No,Y
0,1,0
1,2,0
2,3,0
3,4,0
4,5,1


In [50]:
solution.to_csv("solution_DTClassifier.csv", index=False)

### random forest

In [51]:
from sklearn.ensemble import RandomForestClassifier
RFClassifier = RandomForestClassifier(n_estimators = 10, random_state = 0, max_depth=3)

C:\Users\hp\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [52]:
model=RFClassifier
classficationModelAccuracies(X, y, model)

C:\Users\hp\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys


,train,test
dataSize,15120.0000,6480.0000
accuracy,0.8162,0.8083
tpPct,0.7159,0.7026
fpPct,0.1738,0.1819


In [53]:
predited_values=model.predict(test)
solution=pd.DataFrame()
solution["Ref.No"]=list(range(1,2400))
solution["Y"]=list(predited_values)

In [54]:
solution.to_csv("solution_RFC.csv", index=False)

In [ ]:
def VIF(X):
    from statsmodels.stats.outliers_influence import variance_inflation_factor
    data_mat=X.as_matrix()
    vif=[variance_inflation_factor(data_mat,i) for i in range(data_mat.shape[1])]
    vif_factor = pd.DataFrame()
    vif_factor["columns"] = X.columns
    vif_factor["vif"] = vif
    return(vif_factor)

In [ ]:
vif_df=VIF(X)

#### In an iterative way takeout all VIF values greater than 5

In [ ]:
while vif_df.vif.max()>5:
    drop_column=list(vif_df[vif_df.vif==vif_df.vif.max()]['columns'].values)
    X.drop(drop_column,axis=1, inplace=True)
    vif_df=VIF(X)
vif_df

In [ ]:
test=test.loc[:,['credit_amount', 'education', 'marital_status', 'sep_hist', 'aug_hist']]

# Adding extra columns

In [8]:
citi = pd.read_csv('C:\\Users\\hp\\Desktop\\Great Lakes\\Supervised Learning\\Kaggle Competetion\\Knn\\train.csv')
citi.columns

Index(['Ref.No', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10',
       'X11', 'X12', 'X13', 'X14', 'X15', 'Y'],
      dtype='object')